In [12]:
import torch
from torch.utils.data import DataLoader, random_split

import pytorch_lightning as pl
from pytorch_lightning.callbacks.early_stopping import EarlyStopping

from PIL import Image, ImageDraw
from torchvision import transforms

from dataloader.dataloader import DTSegmentationDataset
from model.model import DTSegmentationNetwork

In [2]:
%load_ext autoreload
%autoreload 2

### Load the dataset

In [4]:
dataset = DTSegmentationDataset()

# Split the dataset into train and test
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train, val = random_split(dataset, [train_size, test_size])
print(f"Train size: {len(train)}, val size: {len(val)}")

Train size: 100, val size: 25


### Overfit Test

In [9]:
hparams = {
    # --- Model ---
    # | Model hyperparameters
    'num_classes': 5,
    # | Optimization hyperparameters
    "learning_rate": 0.0625,
    "weight_decay": 0.000000625,
    "lr_decay": 0.25,
    
    # --- Dataloader (Hardware-specific) ---
    "batch_size": 12,
    "num_workers": 2,
}

model = DTSegmentationNetwork(hparams)

# Overfit for testing
early_stop_overfit_callback = EarlyStopping(
    monitor='loss',
    patience=150,
    min_delta=0.0005,
    verbose=True,
    mode='min'
)

trainer = pl.Trainer(
    log_every_n_steps=1,
    max_epochs=150,
    overfit_batches=1,
    callbacks=[early_stop_overfit_callback]
)

# image, target = train[0]
# # print(f'image: {image.shape}, target: {target.shape}')
# result = torch.argmax(target, dim=0)
# print(f'image: {image.shape}, result: {result.shape}, value range: {torch.min(result)}-{torch.max(result)}')

trainer.fit(model, DataLoader(train, shuffle=False, batch_size=1), DataLoader(val, shuffle=False, batch_size=1))

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
`Trainer(overfit_batches=1)` was configured so 1 batch will be used.

  | Name  | Type   | Params
---------------------------------
0 | model | LRASPP | 3.2 M 
---------------------------------
246 K     Trainable params
3.0 M     Non-trainable params
3.2 M     Total params
12.875    Total estimated model params size (MB)


Epoch 0: 100%|██████████| 2/2 [00:00<00:00,  5.46it/s, loss=1.83, v_num=23]

Metric loss improved. New best score: 1.827


Epoch 1: 100%|██████████| 2/2 [00:00<00:00,  4.59it/s, loss=1.11, v_num=23]

Metric loss improved by 1.438 >= min_delta = 0.0005. New best score: 0.389


...

Epoch 54: 100%|██████████| 2/2 [00:00<00:00,  5.21it/s, loss=0.0436, v_num=23]

Metric loss improved by 0.001 >= min_delta = 0.0005. New best score: 0.039


Epoch 149: 100%|██████████| 2/2 [00:00<00:00,  5.09it/s, loss=0.0389, v_num=23]

`Trainer.fit` stopped: `max_epochs=150` reached.


Epoch 149: 100%|██████████| 2/2 [00:00<00:00,  4.75it/s, loss=0.0389, v_num=23]


In [22]:
# visualize the overfitted results
img, target = train[0]
print(f"Target shape: {target.shape}, value range: {torch.min(target)}-{torch.max(target)}")
result = torch.argmax(model(img.unsqueeze(0)), dim=1)
print(f"Result shape: {result.shape}, value range: {torch.min(result)}-{torch.max(result)}")
pil_transform = transforms.ToPILImage()
pil_transform(img).show(title="image")
pil_transform(DTSegmentationDataset.label_img_to_rgb(target)).show(title="ground truth")
pil_transform(DTSegmentationDataset.label_img_to_rgb(result[0])).show(title="prediction")

Target shape: torch.Size([480, 640]), value range: 0-3
target: tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]])
Result shape: torch.Size([1, 480, 640]), value range: 0-3


### Training the model

In [ ]:
#